In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`

import $ivy.$

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark SQL")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 16:28:22 INFO SparkContext: Running Spark version 3.5.5
25/03/13 16:28:22 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 16:28:22 INFO SparkContext: Java version 11.0.26
25/03/13 16:28:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark: SparkSession = org.apache.spark.sql.SparkSession@7ec03e09
import spark.implicits._

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.5.5


In [6]:
val customerDF = spark.read.format("json").load("data/customer_data.json")
customerDF.show(5, false)

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

customerDF: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [7]:
customerDF.createOrReplaceTempView("customer")

In [8]:
spark.sql("select Birthdate, Country from customer")

res8: DataFrame = [Birthdate: string, Country: string]

In [9]:
spark.sql("select Birthdate, Country from customer").show(5, false)

+-------------------+--------------+
|Birthdate          |Country       |
+-------------------+--------------+
|1994-02-20 00:46:27|United Kingdom|
|1988-06-21 00:15:34|Iceland       |
|1974-11-26 15:30:20|Finland       |
|1977-05-06 23:57:35|Italy         |
|1996-09-13 19:14:27|Norway        |
+-------------------+--------------+
only showing top 5 rows



In [10]:
spark.sql("select Birthdate as Date, Country from customer").show(5, false)

+-------------------+--------------+
|Date               |Country       |
+-------------------+--------------+
|1994-02-20 00:46:27|United Kingdom|
|1988-06-21 00:15:34|Iceland       |
|1974-11-26 15:30:20|Finland       |
|1977-05-06 23:57:35|Italy         |
|1996-09-13 19:14:27|Norway        |
+-------------------+--------------+
only showing top 5 rows



In [11]:
spark.sql("select Birthdate as Date, Country, true as Flag from customer").show(5, false)

+-------------------+--------------+----+
|Date               |Country       |Flag|
+-------------------+--------------+----+
|1994-02-20 00:46:27|United Kingdom|true|
|1988-06-21 00:15:34|Iceland       |true|
|1974-11-26 15:30:20|Finland       |true|
|1977-05-06 23:57:35|Italy         |true|
|1996-09-13 19:14:27|Norway        |true|
+-------------------+--------------+----+
only showing top 5 rows



In [12]:
spark.sql("select Birthdate as Date, Country, true as Flag from customer").printSchema

root
 |-- Date: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Flag: boolean (nullable = false)



In [13]:
spark.sql("select * from customer where Country = 'Norway'").show(5, false)

+--------------------------------------------------+-------------------+-------+----------+-------------------------+--------------+---------------+
|Address                                           |Birthdate          |Country|CustomerID|Email                    |Name          |Username       |
+--------------------------------------------------+-------------------+-------+----------+-------------------------+--------------+---------------+
|17677 Mark Crest\nWalterberg, IA 39017            |1996-09-13 19:14:27|Norway |12350     |amyholland@yahoo.com     |Natalie Ford  |gregoryharrison|
|50047 Smith Point Suite 162\nWilkinsstad, PA 04106|1969-06-21 03:39:20|Norway |12352     |vcarter@hotmail.com      |Dana Clarke   |hmyers         |
|0297 Jacob Ranch Apt. 019\nNorth Judith, NV 27455 |1990-06-01 14:49:52|Norway |12381     |douglaschavez@hotmail.com|Matthew Jones |stephanie68    |
|3102 Hopkins Walk\nAndrechester, MD 54461         |1976-06-19 08:10:24|Norway |12430     |crystalromero@h

In [14]:
spark.sql("select * from customer where Country <> 'Iceland'").show(5, false)

+--------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                           |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+--------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                  |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|Unit 2676 Box 9352\nDPO AA 38560                  |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, CT 53165         |1977-05-06 23:57:35|Italy         |12349     |dustin37@yahoo.com      |Brad Cardenas  |charleshudson   |
|17677 Mark Crest\nWalterberg, IA 39017            |1996-0

In [15]:
spark.sql("select * from customer order by CustomerID desc").show(5, false)

+----------------------------------------------------+-------------------+--------------+----------+-----------------------+--------------------+-----------------+
|Address                                             |Birthdate          |Country       |CustomerID|Email                  |Name                |Username         |
+----------------------------------------------------+-------------------+--------------+----------+-----------------------+--------------------+-----------------+
|6942 Connie Skyway\nPatrickville, WA 16551          |1973-10-24 00:52:10|United Kingdom|12989     |amber97@hotmail.com    |Brandon Contreras   |ecasey           |
|79375 David Neck\nWest Matthewton, NJ 92863         |1971-05-04 22:20:10|United Kingdom|12988     |erica98@gmail.com      |Gabriel Romero      |qknight          |
|00881 West Flat\nNorth Emily, IL 32130              |1997-03-05 19:20:57|United Kingdom|12987     |vkeith@yahoo.com       |Christopher Lawrence|smcintyre        |
|499 Jonathan St

In [16]:
spark.sql("select Birthdate, date_format(Birthdate, 'yyyy-MM-dd') as Date from customer").show(5, false)

+-------------------+----------+
|Birthdate          |Date      |
+-------------------+----------+
|1994-02-20 00:46:27|1994-02-20|
|1988-06-21 00:15:34|1988-06-21|
|1974-11-26 15:30:20|1974-11-26|
|1977-05-06 23:57:35|1977-05-06|
|1996-09-13 19:14:27|1996-09-13|
+-------------------+----------+
only showing top 5 rows



In [17]:
spark.sql("select Name, Username, array(Name, Username) as Identity from customer").show(5, false)

+---------------+----------------+---------------------------------+
|Name           |Username        |Identity                         |
+---------------+----------------+---------------------------------+
|Lindsay Cowan  |valenciajennifer|[Lindsay Cowan, valenciajennifer]|
|Katherine David|hillrachel      |[Katherine David, hillrachel]    |
|Leslie Martinez|serranobrian    |[Leslie Martinez, serranobrian]  |
|Brad Cardenas  |charleshudson   |[Brad Cardenas, charleshudson]   |
|Natalie Ford   |gregoryharrison |[Natalie Ford, gregoryharrison]  |
+---------------+----------------+---------------------------------+
only showing top 5 rows



In [18]:
spark.sql("select Country, count(*) as count from customer group by Country order by count desc").show(5, false)

+--------------+-----+
|Country       |count|
+--------------+-----+
|United Kingdom|124  |
|Germany       |86   |
|France        |86   |
|Spain         |30   |
|Belgium       |25   |
+--------------+-----+
only showing top 5 rows



In [19]:
spark.sql("""
select
  Country
  , date_format(Birthdate, 'yyyy-MM-dd') as bd
  , min(CustomerID)
  , max(CustomerID)
from customer
group by Country, bd
order by Country""")
.show(5, false)

+---------+----------+---------------+---------------+
|Country  |bd        |min(CustomerID)|max(CustomerID)|
+---------+----------+---------------+---------------+
|Australia|1966-09-17|12422          |12422          |
|Australia|1967-11-29|12424          |12424          |
|Australia|1985-12-30|12386          |12386          |
|Australia|1986-08-28|12434          |12434          |
|Australia|1987-02-26|12393          |12393          |
+---------+----------+---------------+---------------+
only showing top 5 rows



In [20]:
spark.sql("select count(*) from customer").show

+--------+
|count(1)|
+--------+
|     507|
+--------+



In [21]:
spark.sql("""
select count(*) from (
  select * from customer
  union all
  select * from customer
)""").show

+--------+
|count(1)|
+--------+
|    1014|
+--------+



In [22]:
val retailDF = spark.read.format("json").load("data/retail_data.json")

retailDF: DataFrame = [CustomerID: string, Description: string ... 5 more fields]

In [23]:
retailDF.createOrReplaceTempView("retail")

In [24]:
val jDF = spark.sql("""
select c.*, r.*
from customer c
join retail r
on c.CustomerID == r.CustomerID
""")

jDF: DataFrame = [Address: string, Birthdate: string ... 12 more fields]

In [25]:
jDF.printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- UnitPrice: string (nullable = true)



In [26]:
jDF.show(5, false)

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+----------+---------------------------------+---------------+---------+--------+---------+---------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |CustomerID|Description                      |InvoiceDate    |InvoiceNo|Quantity|StockCode|UnitPrice|
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+----------+---------------------------------+---------------+---------+--------+---------+---------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|12346     |MEDIUM CERAMIC TOP STORAGE JAR   |1/18/2011 10

In [27]:
jDF.select("CustomerID", "Name", "Birthdate", "InvoiceDate").show(5, false)

org.apache.spark.sql.AnalysisException: [AMBIGUOUS_REFERENCE] Reference `CustomerID` is ambiguous, could be: [`c`.`CustomerID`, `r`.`CustomerID`].

In [28]:
val jDF2 = spark.sql("""
select c.CustomerID, c.Name, c.Birthdate, r.InvoiceDate
from customer c
join retail r
on c.CustomerID == r.CustomerID
""")

jDF2: DataFrame = [CustomerID: string, Name: string ... 2 more fields]

In [29]:
jDF2.show(5, false)

+----------+---------------+-------------------+---------------+
|CustomerID|Name           |Birthdate          |InvoiceDate    |
+----------+---------------+-------------------+---------------+
|12346     |Lindsay Cowan  |1994-02-20 00:46:27|1/18/2011 10:01|
|12346     |Lindsay Cowan  |1994-02-20 00:46:27|1/18/2011 10:17|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
+----------+---------------+-------------------+---------------+
only showing top 5 rows



In [30]:
jDF2.select("CustomerID", "Name", "Birthdate", "InvoiceDate").show(5, false)

+----------+---------------+-------------------+---------------+
|CustomerID|Name           |Birthdate          |InvoiceDate    |
+----------+---------------+-------------------+---------------+
|12346     |Lindsay Cowan  |1994-02-20 00:46:27|1/18/2011 10:01|
|12346     |Lindsay Cowan  |1994-02-20 00:46:27|1/18/2011 10:17|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
+----------+---------------+-------------------+---------------+
only showing top 5 rows

